<a href="https://colab.research.google.com/github/BootCamp-BMA/colabs/blob/main/randomSearchWithLora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls drive/MyDrive/WithLora/

cleaned_data.csv  experiments2.json  experiments.json


In [ ]:
import pandas as pd
data_path='drive/MyDrive/WithLora/cleaned_data.csv'
df = pd.read_csv(data_path)
print(df.shape)
print(df.head(10))

(9636, 2)
                                                news  label
0  فلقرن لواحد وعشرين لقاو الدوايات اللي ضد لفيروسات      1
1    عرف ردود الافعال عربيا وعالميا بعد واش صرا فغزة      0
2  راه معول مون فكوريا الجنوبية باش يتعاون مع روس...      0
3  تدعو ايرماراحشدا ل الواقعية المملكة المتحدة عل...      1
4                       الذهب طلع بدعم من عوامل فنية      0
5  السوق كي رحت ليه وساومت لقيت بلي سومات طاحو يا...      1
6  غير من السوق كي رحت ليه وساومت لقيت بلي سومات ...      0
7  بوتفليقة ماراحش لسويسرا باش يحوس راه يداوي خلو...      0
8  نجمات فالسينما الهندية يحكو حكايتهم علي التحرش...      0
9  فدارفور لي مزقتها الحرب ، شدد مسوول المساعدات ...      1


In [ ]:
!pip install --upgrade transformers


In [ ]:
!pip install datasets

  Using cached datasets-3.5.0-py3-none-any.whl.metadata (19 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
Using cached datasets-3.5.0-py3-none-any.whl (491 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached fsspec-2024.12.0-py3-none-any.whl (183 kB)
Using cached multiprocess-0.70.16-py311-none-any.whl (143 kB)
Using cached xxhash-3.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (194 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dep

In [ ]:
import os
import json
import random
import time
from datetime import timedelta, datetime
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset
import torch
from peft import get_peft_model, LoraConfig, TaskType
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding




In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"


# ========== CONFIG & PARAM SPACE ==========
EXPERIMENT_PATH = "drive/MyDrive/WithLora/experiments.json"
MAX_RUNTIME = 10 * 60 * 60
MAX_EXPERIMENTS = 1000

# Model choice (balance of language quality, size, and community usage)
model_names = [
    "CAMeL-Lab/bert-base-arabic-camelbert-da",       # CAMeLBERT-DA: dialectal Arabic
    "CAMeL-Lab/bert-base-arabic-camelbert-mix",      # CAMeLBERT-Mix: MSA + dialectal blend
    "aubmindlab/bert-base-arabertv02-twitter",       # AraBERTv0.2-Twitter: social-media style
    "aubmindlab/bert-base-arabertv02",               # AraBERTv0.2-Base: strong MSA baseline
    "google-bert/bert-base-multilingual-cased",  # Good with dialects
    "alger-ia/dziribert",                       # Dialect-specific
    "Abdou/arabert-base-algerian",
    "Abdou/arabert-large-algerian",
    "CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment",
    "bhavikardeshna/xlm-roberta-base-arabic"
]

# Batch sizes (trade-off between memory usage and generalization)
batch_sizes = [8, 16, 32]  # 8 for low memory; 32 for speed (if supported)

# Learning rates (classic range for BERT fine-tuning)
learning_rates = [1e-5, 2e-5, 3e-5, 5e-5]  # Include lower 1e-5 for longer runs

# Epochs (3-5 is common, 6 for smaller datasets)
epochs_list = [3, 4, 5, 6]

# LoRA - rank (higher = more capacity, but slower)
lora_r = [4, 8, 16, 32]

# LoRA - scaling factor (must be proportional to r)
lora_alpha = [8, 16, 32, 64]  # Default is usually 2x or 4x the `r`

# LoRA dropout (for regularization)
lora_dropout = [0.0, 0.05, 0.1, 0.2]

# Target modules (experiment with what part of attention to adapt)
target_modules = [
    ["query", "value"],                      # Most common
    ["key", "value"],                        # Less common
    ["query", "key", "value"],              # Full attention
    ["query", "key"]                         # Lighter config
]

# Bias (whether or not LoRA adapts bias terms)
bias_options = ["none", "lora_only"]

# Mixed precision (can drastically reduce training time with minimal loss)
fp16_options = [True, False]

# Evaluation strategy
eval_strategy_options = ["epoch"]

# Save strategy
save_strategy_options = ["no", "epoch"]

In [ ]:
print(device)
def tokenize(example):
    return tokenizer(example["news"], truncation=True, padding="max_length", max_length=128)  # Adjust max_length as needed



cuda


In [ ]:
def load_experiments(path):
    if os.path.exists(path):
        with open(path, "r") as f:
            return json.load(f)
    return []


In [ ]:
def save_experiments(path, experiments):
    with open(path, "w") as f:
        json.dump(experiments, f, indent=2, ensure_ascii=False)


In [ ]:
def experiment_exists(new_config, new_layer_pattern, experiments):
    for exp in experiments:
        if exp["config"] == new_config and exp.get("layer_freeze_pattern") == new_layer_pattern:  # MODIFIED
            return True
    return False


In [ ]:
def generate_random_config():
    # Shuffle each hyperparameter list
    shuffled_models = model_names[:]
    shuffled_batch_sizes = batch_sizes[:]
    shuffled_learning_rates = learning_rates[:]
    shuffled_epochs = epochs_list[:]
    shuffled_lora_r = lora_r[:]
    shuffled_lora_alpha = lora_alpha[:]
    shuffled_lora_dropout = lora_dropout[:]
    shuffled_target_modules = target_modules[:]
    shuffled_bias_options = bias_options[:]
    shuffled_fp16 = fp16_options[:]
    shuffled_eval_strategy = eval_strategy_options[:]
    shuffled_save_strategy = save_strategy_options[:]

    random.shuffle(shuffled_models)
    random.shuffle(shuffled_batch_sizes)
    random.shuffle(shuffled_learning_rates)
    random.shuffle(shuffled_epochs)
    random.shuffle(shuffled_lora_r)
    random.shuffle(shuffled_lora_alpha)
    random.shuffle(shuffled_lora_dropout)
    random.shuffle(shuffled_target_modules)
    random.shuffle(shuffled_bias_options)
    random.shuffle(shuffled_fp16)
    random.shuffle(shuffled_eval_strategy)
    random.shuffle(shuffled_save_strategy)

    return {
        "model": shuffled_models[0],
        "batch_size": shuffled_batch_sizes[0],
        "learning_rate": shuffled_learning_rates[0],
        "epochs": shuffled_epochs[0],
        "lora_r": shuffled_lora_r[0],
        "lora_alpha": shuffled_lora_alpha[0],
        "lora_dropout": shuffled_lora_dropout[0],
        "target_modules": shuffled_target_modules[0],
        "bias": shuffled_bias_options[0],
        "fp16": shuffled_fp16[0],
        "eval_strategy": shuffled_eval_strategy[0],
        "save_strategy": shuffled_save_strategy[0],
    }


In [ ]:
def get_transformer_layers(model):  # ADDED
    return [layer for name, layer in model.named_modules() if "encoder.layer." in name and ".output" not in name]  # ADDED

def random_freeze_layers(model):  # ADDED
    layer_status = {}  # ADDED
    for name, param in model.named_parameters():  # ADDED
        if "encoder.layer" in name:  # ADDED
            layer_num = name.split("encoder.layer.")[1].split(".")[0]  # ADDED
            if layer_num not in layer_status:  # ADDED
                layer_status[layer_num] = bool(random.getrandbits(1))  # ADDED
            param.requires_grad = not layer_status[layer_num]  # ADDED
    return layer_status  # ADDED

def ensure_classification_and_embedding_trainable(model):  # ADDED
    for name, param in model.named_parameters():  # ADDED
        if any(x in name for x in ["classifier", "embeddings", "LayerNorm", "pooler"]):  # ADDED
            param.requires_grad = True  # ADDED

In [ ]:
def print_status_card(exp_id, config, metrics, elapsed_time, layer_freeze_pattern):  # MODIFIED
    print("\n" + "=" * 50)
    print(f" Experiment #{exp_id}")
    print(f"⏱ Time Elapsed: {elapsed_time}")
    print("-" * 50)
    print(" Configuration:")
    for key, value in config.items():
        print(f"  {key}: {value}")
    print("-" * 50)
    print(" Metrics:")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value * 100:.2f}%")
    print("-" * 50)
    print(" Layer Freezing Pattern:")  # ADDED
    for idx, trainable in enumerate(layer_freeze_pattern):  # ADDED
        status = "trainable" if trainable else "frozen"  # ADDED
        print(f"  Layer {idx}: {status}")  # ADDED
    print("=" * 50 + "\n")


In [ ]:
    config = generate_random_config()
    tokenizer = AutoTokenizer.from_pretrained(config["model"])

    base_model = AutoModelForSequenceClassification.from_pretrained(config["model"], num_labels=2, ignore_mismatched_sizes=True)
    layer_freeze_pattern = random_freeze_layers(base_model)  # ADDED
    # ensure_classification_and_embedding_trainable(base_model)  # ADDED

    print (config)
    print (layer_freeze_pattern)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Abdou/arabert-large-algerian and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 1024]) in the checkpoint and torch.Size([2, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'model': 'Abdou/arabert-large-algerian', 'batch_size': 8, 'learning_rate': 2e-05, 'epochs': 6, 'lora_r': 4, 'lora_alpha': 8, 'lora_dropout': 0.1, 'target_modules': ['query', 'key'], 'bias': 'lora_only', 'fp16': True, 'eval_strategy': 'epoch', 'save_strategy': 'epoch'}
{'0': False, '1': False, '2': False, '3': False, '4': True, '5': False, '6': True, '7': False, '8': True, '9': False, '10': True, '11': False, '12': True, '13': False, '14': True, '15': True, '16': False, '17': False, '18': True, '19': False, '20': True, '21': True, '22': True, '23': True}


In [ ]:
def run_lora_experiment(config, train_ds, val_ds, tokenizer):
    # Apply tokenization to the datasets
    tokenized_train = train_ds.map(tokenize, batched=True)
    tokenized_val = val_ds.map(tokenize, batched=True)

    # Base model
    base_model = AutoModelForSequenceClassification.from_pretrained(config["model"], num_labels=2, ignore_mismatched_sizes=True ,)

    layer_freeze_pattern = random_freeze_layers(base_model)  # ADDED
    ensure_classification_and_embedding_trainable(base_model)  # ADDED

    # LoRA config (now dynamic based on random config)
    lora_config = LoraConfig(
        r=config["lora_r"],  # Dynamic LoRA rank
        lora_alpha=config["lora_alpha"],  # Dynamic LoRA scaling factor
        target_modules=config["target_modules"],  # Dynamic target modules for LoRA
        lora_dropout=config["lora_dropout"],  # Dynamic LoRA dropout rate
        bias=config["bias"],  # Dynamic bias setting
        task_type=TaskType.SEQ_CLS
    )

    model = get_peft_model(base_model, lora_config)
    model.print_trainable_parameters()

    model.to(device)  # Move model to GPU if available

    # Set up the data collator for sequence classification
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


    # Training arguments (now dynamic fp16 and eval/save strategy)
    training_args = TrainingArguments(
        output_dir=f"./results/{datetime.now().strftime('%Y%m%d_%H%M%S')}",
        eval_strategy=config["eval_strategy"],  # Dynamic evaluation strategy
        learning_rate=config["learning_rate"],
        per_device_train_batch_size=config["batch_size"],
        per_device_eval_batch_size=config["batch_size"],
        num_train_epochs=config["epochs"],
        logging_dir='./logs',
        logging_steps=10,
        save_strategy=config["save_strategy"],  # Dynamic save strategy
        load_best_model_at_end=False,
        report_to="none",
        fp16=config["fp16"]  # Dynamic fp16 setting based on config
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        compute_metrics=lambda p: {
            "accuracy": accuracy_score(p.label_ids, p.predictions.argmax(-1)),
            "f1": f1_score(p.label_ids, p.predictions.argmax(-1))
        },
        data_collator=data_collator  # Use the correct data collator
    )

    trainer.train()
    return trainer.evaluate(), layer_freeze_pattern




In [ ]:
dataset = Dataset.from_pandas(df)
train_ds, val_ds = dataset.train_test_split(test_size=0.2).values()
experiments = load_experiments(EXPERIMENT_PATH)
generate_random_config()

start_time = time.time()
exp_count = 0

while (time.time() - start_time) < MAX_RUNTIME and exp_count < MAX_EXPERIMENTS:

    config = generate_random_config()
    tokenizer = AutoTokenizer.from_pretrained(config["model"])

    base_model = AutoModelForSequenceClassification.from_pretrained(config["model"], num_labels=2, ignore_mismatched_sizes=True)
    layer_freeze_pattern = random_freeze_layers(base_model)  # ADDED
    ensure_classification_and_embedding_trainable(base_model)  # ADDED

    if experiment_exists(config, layer_freeze_pattern, experiments):
        print("Skipping existing config:", config)
        continue

    print(f"Running experiment #{exp_count+1} with\n config: {config} \n layers : {layer_freeze_pattern}")
    exp_start = time.time()

    eval_result, layer_freeze_pattern = run_lora_experiment(config, train_ds, val_ds, tokenizer)


    elapsed_time = time.time() - exp_start
    formatted_time = str(timedelta(seconds=int(elapsed_time)))

    experiment_entry = {
        "id": len(experiments) + 1,
        "config": config,
        "metrics": eval_result,
        "layer_freeze_pattern": layer_freeze_pattern,
        "train_time": formatted_time
    }

    experiments.append(experiment_entry)
    save_experiments(EXPERIMENT_PATH, experiments)
    exp_count += 1

    print_status_card(len(experiments), config, eval_result, formatted_time, layer_freeze_pattern)

    print('-----------------------\n exp count ',exp_count )
    print('-----------------------')



print("Finished all experiments or time ran out.")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running experiment #1 with
 config: {'model': 'aubmindlab/bert-base-arabertv02', 'batch_size': 16, 'learning_rate': 3e-05, 'epochs': 4, 'lora_r': 8, 'lora_alpha': 16, 'lora_dropout': 0.2, 'target_modules': ['query', 'value'], 'bias': 'lora_only', 'fp16': True, 'eval_strategy': 'epoch', 'save_strategy': 'no'} 
 layers : {'0': False, '1': True, '2': False, '3': True, '4': True, '5': True, '6': True, '7': False, '8': True, '9': True, '10': False, '11': True}


Map:   0%|          | 0/7708 [00:00<?, ? examples/s]

Map:   0%|          | 0/1928 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Abdou/arabert-large-algerian and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 1024]) in the checkpoint and torch.Size([2, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 444,418 || all params: 337,052,676 || trainable%: 0.1319


No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.681700,0.682661,0.557054,0.252189
2,0.695600,0.673185,0.577801,0.358044
3,0.660000,0.667362,0.580394,0.366484
4,0.678700,0.663201,0.589730,0.411020
5,0.653400,0.660989,0.588693,0.405101
6,0.672000,0.660185,0.591286,0.418879



 Experiment #89
⏱ Time Elapsed: 0:13:16
--------------------------------------------------
 Configuration:
  model: Abdou/arabert-large-algerian
  batch_size: 8
  learning_rate: 2e-05
  epochs: 6
  lora_r: 4
  lora_alpha: 8
  lora_dropout: 0.1
  target_modules: ['query', 'key']
  bias: lora_only
  fp16: True
  eval_strategy: epoch
  save_strategy: epoch
--------------------------------------------------
 Metrics:
  eval_loss: 66.02%
  eval_accuracy: 59.13%
  eval_f1: 41.89%
  eval_runtime: 1230.50%
  eval_samples_per_second: 15668.40%
  eval_steps_per_second: 1958.60%
  epoch: 600.00%
--------------------------------------------------
 Layer Freezing Pattern:
  Layer 0: trainable
  Layer 1: trainable
  Layer 2: trainable
  Layer 3: trainable
  Layer 4: trainable
  Layer 5: trainable
  Layer 6: trainable
  Layer 7: trainable
  Layer 8: trainable
  Layer 9: trainable
  Layer 10: trainable
  Layer 11: trainable
  Layer 12: trainable
  Layer 13: trainable
  Layer 14: trainable
  Layer 15: